Here we continue to investigate the corpus collected from regulations.gov for the proposal EPA–R08–OAR–2015–0463

We follow "Topics and Transformations" a follow up to "Corpora_and_Vector_Spaces" gensim tutorial https://radimrehurek.com/gensim/tut2.html



# Topics and Transformation

Don't forget to set

In [1]:
import logging
import os.path

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

if you want to see logging events.

## Transformation interface

In the previous tutorial on [Corpora and Vector Spaces](https://radimrehurek.com/gensim/tut1.html), we created a corpus of documents represented as a stream of vectors. To continue, let’s fire up gensim and use that corpus:

In [2]:
from gensim import corpora, models, similarities
if (os.path.exists("/tmp/dstore.dict")):
    dictionary = corpora.Dictionary.load('/tmp/dstore.dict')
    corpus = corpora.MmCorpus('/tmp/dstore.mm')
    print("Used files generated from NLP 1 (comments)")
else:
    print("Please run first tutorial to generate data set")

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
2017-03-07 12:49:27,721 : INFO : 'pattern' package not found; tag filters are not available for English
2017-03-07 12:49:27,739 : INFO : loading Dictionary object from /tmp/dstore.dict
2017-03-07 12:49:27,800 : INFO : loaded /tmp/dstore.dict
2017-03-07 12:49:27,804 : INFO : loaded corpus index from /tmp/dstore.mm.index
2017-03-07 12:49:27,807 : INFO : initializing corpus reader from /tmp/dstore.mm
2017-03-07 12:49:27,810 : INFO : accepted corpus with 23 documents, 22734 features, 47941 non-zero entries


Used files generated from NLP 1 (comments)


In [3]:
print (dictionary[0])
print (dictionary[1])
print (dictionary[2])

limited
unscientific
mfrs


In this tutorial, I will show how to transform documents from one vector representation into another. This process serves two goals:

1. To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and (hopefully) more semantic way.
1. To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction).

### Creating a transformation

The transformations are standard Python objects, typically initialized by means of a training corpus:

In [4]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2017-03-07 12:49:39,566 : INFO : collecting document frequencies
2017-03-07 12:49:39,714 : INFO : PROGRESS: processing document #0
2017-03-07 12:49:40,569 : INFO : calculating IDF weights for 23 documents and 22733 features (47941 matrix non-zeros)


We used our old corpus from tutorial 1 to initialize (train) the transformation model. Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of going through the supplied corpus once and computing document frequencies of all its features. Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and, consequently, takes much more time.

> <B>Note</B>:
> Transformations always convert between two specific vector spaces. The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

In [5]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.4820409969713478), (1, 0.8761486616087872)]


Or to apply a transformation to a whole corpus:

In [6]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.013769287852095276), (1, 0.004171133160428567), (2, 0.0013903777201428557), (3, 0.0013903777201428557), (4, 0.0011595548090475513), (5, 0.0023191096180951026), (6, 0.007818770993648328), (7, 0.010718380306413682), (8, 0.01469889675403424), (9, 0.0027807554402857113), (10, 0.002294881308682546), (11, 0.0011595548090475513), (12, 0.0011595548090475513), (13, 0.0034786644271426535), (14, 0.024073794121486174), (15, 0.0007649604362275153), (16, 0.0023191096180951026), (17, 0.0007649604362275153), (18, 0.0011595548090475513), (19, 0.013544112764206612), (20, 0.0004198998430929057), (21, 0.0027807554402857113), (22, 0.0029873500419684593), (23, 0.0011595548090475513), (24, 0.00199156669464564), (25, 0.00199156669464564), (26, 0.028956870796757978), (27, 0.0023191096180951026), (28, 0.0015299208724550307), (29, 0.00434376166313796), (30, 0.012945183515196657), (31, 0.00398313338929128), (32, 0.0013903777201428557), (33, 0.00099578334732282), (34, 0.011059338591265334), (35, 0.002319109

In this particular case, we are transforming the same corpus that we used for training, but this is only incidental. Once the transformation model has been initialized, it can be used on any vectors (provided they come from the same vector space, of course), even if they were not used in the training corpus at all. This is achieved by a process called folding-in for LSA, by topic inference for LDA etc.

> <b>Note:</b> 
> Calling model[corpus] only creates a wrapper around the old corpus document stream – actual conversions are done on-the-fly, during document iteration. We cannot convert the entire corpus at the time of calling corpus_transformed = model[corpus], because that would mean storing the result in main memory, and that contradicts gensim’s objective of memory-indepedence. If you will be iterating over the transformed corpus_transformed multiple times, and the transformation is costly, serialize the resulting corpus to disk first and continue using that.

Transformations can also be serialized, one on top of another, in a sort of chain:

In [7]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-03-07 12:51:37,838 : INFO : using serial LSI version on this node
2017-03-07 12:51:37,842 : INFO : updating model with new documents
2017-03-07 12:51:38,892 : INFO : preparing a new chunk of documents
2017-03-07 12:51:38,926 : INFO : using 100 extra samples and 2 power iterations
2017-03-07 12:51:38,927 : INFO : 1st phase: constructing (22734, 102) action matrix
2017-03-07 12:51:38,980 : INFO : orthonormalizing (22734, 102) action matrix
2017-03-07 12:51:42,852 : INFO : 2nd phase: running dense svd on (102, 23) matrix
2017-03-07 12:51:42,896 : INFO : computing the final decomposition
2017-03-07 12:51:42,898 : INFO : keeping 2 factors (discarding 74.877% of energy spectrum)
2017-03-07 12:51:42,944 : INFO : processed documents up to #23
2017-03-07 12:51:42,950 : INFO : topic #0(1.962): 0.365*"bart" + 0.156*"alternative" + 0.145*"0" + 0.141*"rh" + 0.115*"calpuff" + 0.113*"sip" + 0.111*"ucare" + 0.104*"visibility" + 0.101*"3" + 0.095*"2"
2017-03-07 12:51:42,952 : INFO : topic #1(1.389

Here we transformed our Tf-Idf corpus via [Latent Semantic Indexing](http://en.wikipedia.org/wiki/Latent_semantic_indexing) into a latent 2-D space (2-D because we set num_topics=2). Now you’re probably wondering: what do these two latent dimensions stand for? Let’s inspect with models.LsiModel.print_topics():

In [8]:
lsi.print_topics(2)

2017-03-07 12:52:07,001 : INFO : topic #0(1.962): 0.365*"bart" + 0.156*"alternative" + 0.145*"0" + 0.141*"rh" + 0.115*"calpuff" + 0.113*"sip" + 0.111*"ucare" + 0.104*"visibility" + 0.101*"3" + 0.095*"2"
2017-03-07 12:52:07,005 : INFO : topic #1(1.389): 0.245*"ucare" + 0.172*"our" + -0.166*"bart" + 0.159*"recreation" + 0.143*"cid" + 0.136*"mcgrath" + 0.132*"outdoor" + 0.130*"societal" + 0.117*"my" + 0.117*"shaun"


[(0,
  u'0.365*"bart" + 0.156*"alternative" + 0.145*"0" + 0.141*"rh" + 0.115*"calpuff" + 0.113*"sip" + 0.111*"ucare" + 0.104*"visibility" + 0.101*"3" + 0.095*"2"'),
 (1,
  u'0.245*"ucare" + 0.172*"our" + -0.166*"bart" + 0.159*"recreation" + 0.143*"cid" + 0.136*"mcgrath" + 0.132*"outdoor" + 0.130*"societal" + 0.117*"my" + 0.117*"shaun"')]

(the topics are printed to log – see the note at the top of this page about activating logging)

It appears that according to LSI, “trees”, “graph” and “minors” are all related words (and contribute the most to the direction of the first topic), while the second topic practically concerns itself with all the other words. As expected, the first five documents are more strongly related to the second topic while the remaining four documents to the first topic:

In [9]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, 0.83271110508447355), (1, -0.30340369352077917)]
[(0, 0.21656614611379407), (1, -0.021384138994083805)]
[(0, 0.7409610129960682), (1, -0.24767205808454554)]
[(0, 0.020451898603658814), (1, 0.011227288258037576)]
[(0, 0.83443815243570851), (1, -0.3044436032549705)]
[(0, 0.18510077332981975), (1, 0.090289621070501341)]
[(0, 0.37765833790111497), (1, 0.41212423923896374)]
[(0, 0.37610936122850386), (1, 0.42220609202352294)]
[(0, 0.60937985863865529), (1, -0.17737249437118868)]
[(0, 0.16845878973183254), (1, 0.09554140727042372)]
[(0, 0.35600376879257845), (1, -0.059286853533599929)]
[(0, 0.47102080221256659), (1, -0.15860698221060288)]
[(0, 0.23498705969064751), (1, 0.035083402865322161)]
[(0, 0.17512709529317372), (1, 0.21529097774827791)]
[(0, 0.51378856022775488), (1, 0.0078224189021609426)]
[(0, 0.22667148695919334), (1, 0.36745036717740959)]
[(0, 0.19862814506707357), (1, 0.49140583446519087)]
[(0, 0.18897107734416543), (1, 0.48017571789921737)]
[(0, 0.22156594793727905), (1, 0.

In [10]:
lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('/tmp/model.lsi')

2017-03-07 11:36:14,555 : INFO : saving Projection object under /tmp/model.lsi.projection, separately None
2017-03-07 11:36:14,569 : INFO : saved /tmp/model.lsi.projection
2017-03-07 11:36:14,571 : INFO : saving LsiModel object under /tmp/model.lsi, separately None
2017-03-07 11:36:14,573 : INFO : not storing attribute projection
2017-03-07 11:36:14,574 : INFO : not storing attribute dispatcher
2017-03-07 11:36:14,718 : INFO : saved /tmp/model.lsi
2017-03-07 11:36:14,721 : INFO : loading LsiModel object from /tmp/model.lsi
2017-03-07 11:36:14,825 : INFO : loading id2word recursively from /tmp/model.lsi.id2word.* with mmap=None
2017-03-07 11:36:14,830 : INFO : setting ignored attribute projection to None
2017-03-07 11:36:14,833 : INFO : setting ignored attribute dispatcher to None
2017-03-07 11:36:14,836 : INFO : loaded /tmp/model.lsi
2017-03-07 11:36:14,839 : INFO : loading LsiModel object from /tmp/model.lsi.projection
2017-03-07 11:36:14,843 : INFO : loaded /tmp/model.lsi.projection


The next question might be: just how exactly similar are those documents to each other? Is there a way to formalize the similarity, so that for a given input document, we can order some other set of documents according to their similarity? Similarity queries are covered in the [next tutorial](https://radimrehurek.com/gensim/tut3.html).

## Available transformations

Gensim implements several popular Vector Space Model algorithms:

### [Term Frequency * Inverse Document Frequency](http://en.wikipedia.org/wiki/Tf–idf) 
Tf-Idf expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality, except that features which were rare in the training corpus will have their value increased. It therefore converts integer-valued vectors into real-valued ones, while leaving the number of dimensions intact. It can also optionally normalize the resulting vectors to (Euclidean) unit length.

In [11]:
model = models.TfidfModel(corpus, normalize=True)

2017-03-07 11:36:19,378 : INFO : collecting document frequencies
2017-03-07 11:36:19,530 : INFO : PROGRESS: processing document #0
2017-03-07 11:36:20,320 : INFO : calculating IDF weights for 23 documents and 22733 features (47941 matrix non-zeros)


### [Latent Semantic Indexing, LSI (or sometimes LSA)](http://en.wikipedia.org/wiki/Latent_semantic_indexing) 
LSI transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality. For the toy corpus above we used only 2 latent dimensions, but on real corpora, target dimensionality of 200–500 is recommended as a “golden standard” [1].

In [12]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2017-03-07 11:36:27,223 : INFO : using serial LSI version on this node
2017-03-07 11:36:27,227 : INFO : updating model with new documents
2017-03-07 11:36:28,234 : INFO : preparing a new chunk of documents
2017-03-07 11:36:28,265 : INFO : using 100 extra samples and 2 power iterations
2017-03-07 11:36:28,267 : INFO : 1st phase: constructing (22734, 400) action matrix
2017-03-07 11:36:28,470 : INFO : orthonormalizing (22734, 400) action matrix
2017-03-07 11:36:50,300 : INFO : 2nd phase: running dense svd on (400, 23) matrix
2017-03-07 11:36:50,498 : INFO : computing the final decomposition
2017-03-07 11:36:50,503 : INFO : keeping 23 factors (discarding 0.000% of energy spectrum)
2017-03-07 11:36:50,814 : INFO : processed documents up to #23
2017-03-07 11:36:50,820 : INFO : topic #0(1.962): 0.365*"bart" + 0.156*"alternative" + 0.145*"0" + 0.141*"rh" + 0.115*"calpuff" + 0.113*"sip" + 0.111*"ucare" + 0.104*"visibility" + 0.101*"3" + 0.095*"2"
2017-03-07 11:36:50,826 : INFO : topic #1(1.389

LSI training is unique in that we can continue “training” at any point, simply by providing more training documents. This is done by incremental updates to the underlying model, in a process called online training. Because of this feature, the input document stream may even be infinite – just keep feeding LSI new documents as they arrive, while using the computed transformation model as read-only in the meanwhile!

> <b>Example</b> 
> 
> model.add_documents(another_tfidf_corpus) # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
> lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model

> model.add_documents(more_documents) # tfidf_corpus + another_tfidf_corpus + more_documents
> lsi_vec = model[tfidf_vec]


See the [gensim.models.lsimodel](https://radimrehurek.com/gensim/models/lsimodel.html#module-gensim.models.lsimodel) documentation for details on how to make LSI gradually “forget” old observations in infinite streams. If you want to get dirty, there are also parameters you can tweak that affect speed vs. memory footprint vs. numerical precision of the LSI algorithm.

gensim uses a novel online incremental streamed distributed training algorithm (quite a mouthful!), which I published in [5]. gensim also executes a stochastic multi-pass algorithm from Halko et al. [4] internally, to accelerate in-core part of the computations. See also 
    [Experiments on the English Wikipedia](https://radimrehurek.com/gensim/wiki.html) for further speed-ups by distributing the computation across a cluster of computers.

### [Random Projections](http://www.cis.hut.fi/ella/publications/randproj_kdd.pdf)
RP aim to reduce vector space dimensionality. This is a very efficient (both memory- and CPU-friendly) approach to approximating TfIdf distances between documents, by throwing in a little randomness. Recommended target dimensionality is again in the hundreds/thousands, depending on your dataset.

In [13]:
model = models.RpModel(corpus_tfidf, num_topics=500)

2017-03-07 12:35:17,671 : INFO : no word id mapping provided; initializing from corpus, assuming identity
2017-03-07 12:35:18,770 : INFO : constructing (500, 22734) random matrix


### [Latent Dirichlet Allocation, LDA](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) 
LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [ ]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=1000)

2017-03-07 12:35:22,828 : INFO : using symmetric alpha at 0.001
2017-03-07 12:35:22,870 : INFO : using symmetric eta at 0.001
2017-03-07 12:35:22,877 : INFO : using serial LDA version on this node
2017-03-07 12:44:07,226 : INFO : running online LDA training, 1000 topics, 1 passes over the supplied corpus of 23 documents, updating model once every 23 documents, evaluating perplexity every 23 documents, iterating 50x with a convergence threshold of 0.001000
2017-03-07 12:44:07,257 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


gensim uses a fast implementation of online LDA parameter estimation based on [2], modified to run in distributed mode on a cluster of computers.

### [Hierarchical Dirichlet Process, HDP](http://jmlr.csail.mit.edu/proceedings/papers/v15/wang11a/wang11a.pdf) 
HDP is a non-parametric bayesian method (note the missing number of requested topics):

In [15]:
model = models.HdpModel(corpus, id2word=dictionary)

gensim uses a fast, online implementation based on [3]. The HDP model is a new addition to gensim, and still rough around its academic edges – use with care.

Adding new VSM transformations (such as different weighting schemes) is rather trivial; see the API reference or directly the Python code for more info and examples.

It is worth repeating that these are all unique, incremental implementations, which do not require the whole training corpus to be present in main memory all at once. With memory taken care of, I am now improving Distributed Computing, to improve CPU efficiency, too. If you feel you could contribute (by testing, providing use-cases or code), please let me know.

Continue on to the next tutorial on [Similarity Queries](./Similarity_Queries.ipynb).

---
[1]	Bradford. 2008. An empirical study of required dimensionality for large-scale latent semantic indexing applications.  
[2]	Hoffman, Blei, Bach. 2010. Online learning for Latent Dirichlet Allocation.  
[3]	Wang, Paisley, Blei. 2011. Online variational inference for the hierarchical Dirichlet process.  
[4]	Halko, Martinsson, Tropp. 2009. Finding structure with randomness.  
[5]	Řehůřek. 2011. Subspace tracking for Latent Semantic Analysis.  